In [ ]:
import pandas as pd
import numpy as np
import datetime

import dask
import dask.dataframe as dd

dfs = []
# Data is split into 79 files, modify the range if you don't want to use all files.
for i in range(0, 9):
    if i < 10:
        i = f"0{i}"
    df = dd.read_csv(f"data/2022_place_canvas_history-0000000000{i}.csv.gzip",
                                    compression="gzip", blocksize=None)#, parse_dates = ["timestamp"])
    dfs.append(df)
df = dd.concat(dfs)
df.head()

Pandas have a built-in data parser but in my experience, it is super slow (in this example it was about 7 times slower).

In [ ]:
del dfs

In [ ]:
def parse_date(date):
    try:
        return datetime.datetime.strptime(date.strip(" UTC"), '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        return datetime.datetime.strptime(date.strip(" UTC"), '%Y-%m-%d %H:%M:%S')

df['timestamp'] = df['timestamp'].map(lambda x: parse_date(x), meta = ('timestamp', 'datetime64[ns]'))

Those cells are split to easily see the **runtime** of the **data parser**.

In [ ]:
df['user_id'] = df['user_id'].astype('string')
df['pixel_color'] = df['pixel_color'].astype('string')

def parse_cord(cord):
    try:
        return float(cord.replace(",", "."))
    except ValueError:
        # Mods sometimes used the rectangle tool to censor NSFW, in those cases cords look like this (x1,y1,x2,y2).
        # With the code below we can get a tuple with opposite rectangle corners, but I didn't need that data.
        ###
        # cords = cord.split('.')
        # return (float(cord[0]+'.'+cord[1]), float(cord[2]+'.'+cord[3]))
        ###
        return np.NaN


df["coordinate"] = df["coordinate"].map(lambda x: parse_cord(x), meta = ("coordinate", float))
df.dropna(how="any")

df.dtypes

### Prepare matplotlib
Each analysis will be split into three parts:
1. Description
2. Data editing
3. Ploting

In [ ]:
import matplotlib.pyplot as plt

plt.style.use('dark_background')
figsize = (20, 10)

#### Pixels per color
We are getting **number of pixels** placed depending on **color** of the **pixel**.

In [ ]:
colors = df["pixel_color"].value_counts().to_frame('counts').reset_index()
colors = colors.rename(columns={'index': 'pixel_color', 'counts': 'counts'}).compute()
colors.head()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.bar(colors["pixel_color"], colors["counts"], color=colors["pixel_color"], edgecolor="#FFFFFF")
ax.set_title('Pixels placed per color')
ax.set_xlabel('color [hex]')
ax.set_ylabel('pixels placed')
plt.xticks(rotation=90)
fig.savefig('./plots/color_counts.pdf')

#### Max pixels placed per user based on color
Same thing as above but we are getting **maximum** number of **pixels** placed by **one user** based on **color**.

In [ ]:
colors = df.iloc[:, 1:3]
colors['count'] = 1
colors = colors.compute()
colors = pd.pivot_table(colors, values='count', index='user_id', columns='pixel_color', aggfunc='count')

color_user = pd.DataFrame(columns=["pixel_color", "user_max"])
for name, column in colors.items():
    temp = pd.DataFrame({"pixel_color": [name], "user_max": [column.max()]})
    color_user = pd.concat([color_user, temp], ignore_index = True)
color_user.sort_values(by='user_max', ascending = False, inplace=True)

In [ ]:
del colors

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
bars = ax.bar(color_user["pixel_color"], color_user["user_max"], color=color_user["pixel_color"], edgecolor="#FFFFFF")
ax.bar_label(bars)
ax.set_title('Max pixels placed per one user based on color')
ax.set_xlabel('color [hex]')
ax.set_ylabel('pixels placed')
plt.xticks(rotation=90)
fig.savefig('./plots/color_user_counts.pdf')

In [ ]:
del color_user

#### Distribution of people who placed *x* many pixels.

In [ ]:
pix_usr = df["user_id"].value_counts().to_frame('pixels').reset_index(drop=True)
# We will use pix_usr later on
pix_usr_count = pix_usr["pixels"].value_counts().to_frame('howmany').compute()
pix_usr_count.head()

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.set_yscale('log')
ax.set_ylabel('user count')
ax.set_xlabel('tiles placed')
ax.set_title('Number of users who placed x many pixels')
ax.hist(pix_usr_count['howmany'])
fig.savefig("./plots/tiles_per_user.pdf")

In [ ]:
del pix_usr_count

#### Top 10 users with maximum pixels placed

In [ ]:
pix_usr = pix_usr.head(10)
pix_usr.index += 1

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.set_ylabel('tiles placed')
ax.set_title('Pixels placed per user [Top 10]')
bars = ax.bar(['#'+str(i) for i in pix_usr.index], pix_usr['pixels'])
ax.bar_label(bars)
ax.set_ylim(ymin=min(pix_usr['pixels']-1))
fig.savefig('./plots/top10_tiles_per_user.pdf')

In [ ]:
del pix_usr

#### Heatmap of pixels
really sketchy but i spend like 7 hours on this plot

In [ ]:
# Creating an array of pixel occurrences
h_map = np.zeros((2000, 2000), dtype='int')
for cord in df["coordinate"]:
    cord = str(cord).split(".")
    h_map[int(cord[0])][int(cord[1])] += 1
print(h_map)

In [ ]:
fig, ax = plt.subplots(figsize=(18.2, 15))
ax = plt.imshow(h_map, interpolation='none', aspect='auto', vmax = 100)
plt.colorbar()
fig.savefig('./plots/pixels_heatmap.pdf')

In [ ]:
del h_map

#### Activity based on time of the day
How many **pixels** were **placed** based on **hour** of the day [UTC].

In [ ]:
hourly = df["timestamp"].compute()
hourly = hourly.dt.strftime('%H')
hourly = hourly.value_counts().rename_axis("hour").to_frame("counts").sort_index()
hourly

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.bar(hourly.index.values, hourly["counts"])
ax.set_title('Pixels placed based on time of the day [UTC]')
ax.set_xlabel('hour of the day')
ax.set_ylabel('pixels placed')
fig.savefig('./plots/hourly_activity.pdf')

In [ ]:
del hourly

#### Activity based the day
How many **pixels** were **placed** based on the **day**.

In [ ]:
daily = df["timestamp"].compute()
daily = daily.dt.strftime('%m-%d')
daily = daily.value_counts().rename_axis("day").to_frame("counts").sort_index()
daily

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.bar(daily.index.values, daily["counts"])
ax.set_title('Pixels placed based on the day')
ax.set_xlabel('date')
ax.set_ylabel('pixels placed')
fig.savefig('./plots/daily_activity.pdf')

In [ ]:
del daily